**1. Mount Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**2. Load YOLOV5**

In [2]:
%cd /content/drive/My Drive/Colab Notebooks/ECS174FP_Task1

/content/drive/My Drive/Colab Notebooks/ECS174FP_Task1


In [3]:
!git clone https://github.com/ultralytics/yolov5.git

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [4]:
%cd yolov5

/content/drive/My Drive/Colab Notebooks/ECS174FP_Task1/yolov5


**3. Install Requirements**

In [5]:
!pip install -r requirements.txt

**4. Train YOLOv5 Model**

In [6]:
!python train.py --img 640 --batch 16 --epochs 300 --data ./dataset.yaml --weights yolov5n.pt

2024-06-08 23:49:22.788350: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 23:49:22.788408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 23:49:22.790436: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5n.pt, cfg=, data=./dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False,

**5. Save Model Files**

In [8]:
from ultralytics import YOLO
model_path = './runs/train/exp6/weights/best.pt'
model_yolo = YOLO(model_path)

**6. Detect and Crop**

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import torch

# Load the YOLOv5 model, assuming you have the custom model trained and stored at the specified path
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp5/weights/best.pt', force_reload=True)

# Specify the image path
img_name="20240607233054.jpg"
img_folder="../test_model/"
img_path = img_folder+img_name
im = Image.open(img_path)

# Perform detection. Resize and augmentations are handled internally by the model.
results = model(im)

# Display the original image with detected bounding boxes
plt.imshow(im)
ax = plt.gca()

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-6-6 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 
WARNING ⚠️ NMS time limit 0.550s exceeded


In [ ]:
# Process detections and save cropped images
num_crops=len(results.xyxy[0])
for i, (*xyxy, conf, cls) in enumerate(results.xyxy[0]):
    xmin, ymin, xmax, ymax = map(int, xyxy)
    # Draw bounding boxes on the original image
    rect = plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, linewidth=2, edgecolor='red', facecolor='none')
    ax.add_patch(rect)

    # Crop the image using the bounding box coordinates
    crop = im.crop((xmin, ymin, xmax, ymax))
    # Save each crop to a file
    crop.save(f'../crops/crop_{img_name[:-4]}_{i}.jpg')  # Save crop with an index label

# Show the plot with bounding boxes
plt.show()

In [ ]:
# Assume you have saved several cropped images named crop_0.jpg, crop_1.jpg, etc.

fig, axes = plt.subplots(1, num_crops, figsize=(15, 5))  # Adjust the subplot grid as needed

# Load and display each cropped image
for i in range(num_crops):
    # Load the cropped image
    crop = Image.open(f'../crops/crop_{img_name[:-4]}_{i}.jpg')
    # Display the cropped image
    axes[i].imshow(crop)
    axes[i].set_title(f'Cropped Image {i}')
    axes[i].axis('off')  # Hide axes ticks

plt.show()